In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from iss_analysis.io import get_mcherry_cells
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import numpy as np
import brainglobe_atlasapi as bga
from pathlib import Path

In [ ]:
arial_font_path = "/nemo/lab/znamenskiyp/home/shared/resources/fonts/arial.ttf"  # update path as needed
arial_prop = fm.FontProperties(fname=arial_font_path)
plt.rcParams["font.family"] = arial_prop.get_name()
fm.fontManager.addfont(arial_font_path)
matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs
main_path = Path("/nemo/lab/znamenskiyp/")

project = "becalia_rabies_barseq"
mouse = "BRAC8498.3e"

# Get the good mCherry cells, i.e. the one for which we have masks and we match the
# starters
mcherry_curated = get_mcherry_cells(
    project, mouse, verbose=True, which="curated", prefix="mCherry_1"
)
# For chamber6, detection fails because of high background, use the manual click instead
mcherry_manual = get_mcherry_cells(
    project, mouse, verbose=True, which="manual", prefix="mCherry_1"
)

m6 = mcherry_manual.query("chamber == 'chamber_06'")
n6 = len(m6)
total = len(mcherry_curated) + n6

print(f"{n6} cells in chamber 6 out of {total}, that's {n6/total*100:.2f}%")

In [ ]:
# Plot manual for chamber 6
m6 = mcherry_manual.query("chamber == 'chamber_06'")
m6 = m6.groupby("roi").aggregate(len).x
plt.plot(m6.index.astype(int) - 10, m6.values, "o-")

# Plot curated for the other
n = 10
for c in ["07", "08", "09", "10"]:
    df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
    d = df.groupby("roi").aggregate(len).x
    plt.plot(d.index + n, d.values, "o-")
    n += d.index.max()

plt.xlabel("Slice index")
plt.ylabel("# of mcherry cells")

In [ ]:
from iss_analysis.io import get_starter_cells, get_sections_info

sec_inf = get_sections_info(project, mouse, chamber=None)
starters_positions = get_starter_cells(project, mouse)

In [ ]:
# Which percentage of the presynaptic cells are in volume as big as our sequenced area
from iss_preprocess.pipeline.ara_registration import load_coordinate_image
from iss_preprocess.io import get_processed_path
from brisc.manuscript_analysis.flatmap_projection import compute_flatmap_coors
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull

from pathlib import Path
from cricksaw_analysis import atlas_utils
from brisc.manuscript_analysis import spatial_plots_rabies as spatial

# Load the area images from in situ sequenced volume
atlas_coverage_path = "becalia_rabies_barseq/BRAC8498.3e/chamber_07/"
area_images = []
for ch in ["07", "08", "09", "10"]:
    for roi in range(1, 11):
        data_path = f"becalia_rabies_barseq/BRAC8498.3e/chamber_{ch}/"
        area_img = load_coordinate_image(
            data_path,
            roi,
            full_scale=False,
        )
        area_images.append(area_img)

barseq_path = get_processed_path("becalia_rabies_barseq").parent.parent
error_correction_ds_name = "BRAC8498.3e_error_corrected_barcodes_26"
barcoded_cells_df = pd.read_pickle(
    barseq_path
    / f"processed/becalia_rabies_barseq/BRAC8498.3e/analysis/{error_correction_ds_name}_cell_barcode_df.pkl"
)
barcoded_cells_df = barcoded_cells_df[barcoded_cells_df["all_barcodes"].notnull()]
barcoded_cells = barcoded_cells_df[["ara_x", "ara_y", "ara_z"]].to_numpy()

# load 2P detected rabies cell coords
project = "rabies_barcoding"
mouse = "BRYC64.2h"
processed = Path("/nemo/lab/znamenskiyp/home/shared/projects/")

points_file = processed / project / mouse / "cellfinder_results_010/points/abc4d.npy"
points = np.load(points_file, allow_pickle=True)
points = points[:, :3] * 0.001  # Nx3 array of XYZ coordinates


def collect_plane_pixels(planes, sample_step: int = 10) -> np.ndarray:
    pix = [P[::sample_step, ::sample_step].reshape(-1, 3) for P in planes]
    return np.concatenate(pix, axis=0)


def in_hull(points: np.ndarray, hull: ConvexHull, tol: float = 1e-12) -> np.ndarray:
    """
    Vectorised point-in-hull test for all cells
    Return boolean mask: True if `points` are inside/on `hull`.
    """
    A, b = hull.equations[:, :3], hull.equations[:, 3]
    return np.all(A @ points.T + b[:, None] <= tol, axis=0)


# Gather pixel points from in situ ara coord images and make a convex hull of the imaged volume
sample_step = 50
pix_coords = collect_plane_pixels(area_images, sample_step)
pix_coords = pix_coords[
    ~np.all(pix_coords == 0, axis=1)
]  # drop (0,0,0) failed atlas points
if len(pix_coords) < 4:
    raise RuntimeError("Not enough valid pixel coordinates to build a hull.")
hull = ConvexHull(pix_coords)
hull_center = pix_coords[hull.vertices].mean(axis=0)
print("Current hull centre:", hull_center)

# Shift the in situ hull so it is centred on the injection site of the 2P data
# 2P 64.2h inj_center in ara coords 8.0, 1.1, 8.2
target_center = np.array([8.0, 1.7307591, 8.533215], dtype=float)
shift_vec = target_center - hull_center
shifted_pix_coords = pix_coords + shift_vec
hull = ConvexHull(shifted_pix_coords)


# Find which 2P detect rabies cells are inside the shifted hull
inside_mask = in_hull(points, hull)
inside_cells = points[inside_mask]
outside_cells = points[~inside_mask]

print(f"Cells inside hull : {inside_cells.shape[0]:,}")
print(f"Cells outside hull: {outside_cells.shape[0]:,}")


flat_coors = compute_flatmap_coors(
    barcoded_cells_df, distance_cutoff=150, projection="top", hemisphere="both"
)
barcoded_cells_df["flatmap_x"] = flat_coors[:, 0]
barcoded_cells_df["flatmap_y"] = flat_coors[:, 1]
barcoded_cells_df["flatmap_z"] = flat_coors[:, 2]

inside_cells_df = pd.DataFrame(inside_cells, columns=["ara_x", "ara_y", "ara_z"])
flat_coors = compute_flatmap_coors(
    inside_cells_df, distance_cutoff=0, projection="top", hemisphere="both"
)
inside_cells_df["flatmap_x"] = flat_coors[:, 0]
inside_cells_df["flatmap_y"] = flat_coors[:, 1]
inside_cells_df["flatmap_z"] = flat_coors[:, 2]

outside_cells_df = pd.DataFrame(outside_cells, columns=["ara_x", "ara_y", "ara_z"])
flat_coors = compute_flatmap_coors(
    outside_cells_df, distance_cutoff=0, projection="top", hemisphere="both"
)
outside_cells_df["flatmap_x"] = flat_coors[:, 0]
outside_cells_df["flatmap_y"] = flat_coors[:, 1]
outside_cells_df["flatmap_z"] = flat_coors[:, 2]

In [ ]:
# Load bin image for coronal area labels
atlas_size = 10
coronal_plane = 801
bin_image = spatial.prepare_area_labels(
    atlas_size=atlas_size,
    xpos=coronal_plane,
    structures=[
        "root",
        "CTX",
        "MB",
        "DG",
        "DG-mo",
        "DG-sg",
        "SCdg",
        "SCdw",
        "SCig",
        "SCiw",
        "SCop",
        "SCsg",
        "SCzo",
        "PAG",
        "MRN",
        "TH",
        "RN",
    ],
)

In [ ]:
from cricksaw_analysis import atlas_utils

bg_atlas = bga.bg_atlas.BrainGlobeAtlas(f"allen_mouse_{atlas_size}um")
atlas = bg_atlas.annotation
dorsal_atlas = atlas_utils.external_view(
    atlas, axis="dorsal", border_only=False, get_index=False, which="first"
)
# Make it contiguous numbers for contours
areas = np.unique(dorsal_atlas)
dorsal_for_borders = np.zeros(dorsal_atlas.shape, dtype="uint8")
for i_area, area in enumerate(areas):
    dorsal_for_borders[dorsal_atlas == area] = i_area

In [ ]:
cor_atlas = atlas[coronal_plane, ...]
coronal_for_borders = np.zeros(cor_atlas.shape, dtype="uint8")
areas = np.unique(cor_atlas)
for i_area, area in enumerate(areas):
    coronal_for_borders[cor_atlas == area] = i_area

In [ ]:
# Add the starter position
color_cells = [
    "dodgerblue",
    "darkorange",
]
rasterized = True
save_fig = True
save_path = main_path / "home/shared/presentations/becalick_2025"
fontsize_dict = {"title": 8, "label": 8, "tick": 6, "legend": 6}
cm = 1 / 2.54

fig = plt.figure(figsize=(12 * cm, 5 * cm), dpi=600)


ax = fig.add_axes([0,0,1,1])
ax.set_xticks([])
ax.set_yticks([])
if True:
    # Proportion of starter in sequencing slices
    ax = fig.add_axes([0.1, 0.2, 0.3, 0.7])
    axt = ax.twinx()
    ax.plot((m6.index.astype(int) - 10) * 20, m6.values, "o-", ms=3, color="grey")
    n = 10

    for c in ["07", "08", "09", "10"]:
        df = mcherry_curated.query(f"chamber == 'chamber_{c}'")
        d = df.groupby("roi").aggregate(len).x
        ax.plot(
            (d.index + n) * 20,
            d.values,
            "o-",
            ms=3,
            color="darkred",
            label="mCherry cells" if c == "10" else "__no_label__",
        )
        sdf = starters_positions.query(f"chamber == 'chamber_{c}'")
        if len(sdf):
            sd = sdf.groupby("roi").aggregate(len).y
            axt.plot(
                (sd.index + n) * 20,
                sd.values,
                "s-",
                ms=3,
                color="dodgerblue",
                label="Starter cells" if c == "10" else "__no_label__",
            )
        n += d.index.max()
    axt.set_ylabel(
        "# of starter cells", color="dodgerblue", fontsize=fontsize_dict["label"]
    )
    ax.set_yticks(np.arange(0, 201, 50))
    ax.set_yticklabels(
        np.arange(0, 201, 50), color="darkred", fontsize=fontsize_dict["tick"]
    )
    ax.set_ylim(0, 200)
    axt.set_yticks(np.arange(0, 81, 20))
    axt.set_yticklabels(
        np.arange(0, 81, 20), color="dodgerblue", fontsize=fontsize_dict["tick"]
    )
    axt.set_ylim(0, 80)
    ax.set_xlabel(r"Slice A/P position ($\mu m$)", fontsize=fontsize_dict["label"])
    ax.set_ylabel(
        "# of mCherry cells", color="darkred", fontsize=fontsize_dict["label"]
    )
    xt = np.arange(0, 1001, 500)
    ax.set_xticks(xt, labels=xt, fontsize=fontsize_dict["tick"])
    ax.spines["top"].set_visible(False)
    axt.spines["top"].set_visible(False)

if True:
    ap_size = 400
    # coronal slice of bulk with seq volume
    if False:
        ax_temp = fig.add_axes([0.43, 0, 0.32, 1])
        ax_temp.set_xticks([])
        ax_temp.set_yticks([])
    ax_coronal = fig.add_axes([0.5, 0, 0.25, 1])

    ax_coronal.contour(
        coronal_for_borders, #bin_image,
        levels=np.arange(0.5, np.max(coronal_for_borders) + 1, 0.5),
        colors="black",
        linewidths=0.1,
        zorder=0,
    )
    cell_plane = outside_cells_df["ara_x"].values * 1000 / atlas_size
    valid = (cell_plane > coronal_plane - (400 / atlas_size)) & (
        cell_plane < coronal_plane + (400 / atlas_size)
    )
    ax_coronal.scatter(
        outside_cells_df.loc[valid, "ara_z"] * 1000 / atlas_size,
        outside_cells_df.loc[valid, "ara_y"] * 1000 / atlas_size,
        s=1,
        edgecolors="none",
        c=color_cells[0],
        zorder=2,
        alpha=0.2,
        rasterized=rasterized,
    )
    cell_plane = inside_cells_df["ara_x"].values * 1000 / atlas_size
    valid = (cell_plane > coronal_plane - (100 / atlas_size)) & (
        cell_plane < coronal_plane + (100 / atlas_size)
    )
    ax_coronal.scatter(
        inside_cells_df.loc[valid, "ara_z"] * 1000 / atlas_size,
        inside_cells_df.loc[valid, "ara_y"] * 1000 / atlas_size,
        s=1,
        edgecolors="none",
        c=color_cells[1],
        zorder=2,
        alpha=0.2,
        rasterized=rasterized,
    )
    ax_coronal.set_aspect("equal")
    ax_coronal.axis("off")
    ax_coronal.invert_yaxis()
    rect = plt.Rectangle((200, 700), 1000 / atlas_size, height=10, color="k")
    ax_coronal.add_artist(rect)

if True:
    # dorsal view slice of bulk with seq volume
    if False:
        ax_dor = fig.add_axes([0.76, 0, 0.3, 1])
        ax_dor.set_xticks([])
        ax_dor.set_yticks([])
    ax_dorsal = fig.add_axes([0.76, 0, 0.25, 1])
    ax_dorsal.contour(
        dorsal_for_borders,
        levels=np.arange(0.5, np.max(dorsal_for_borders) + 1, 0.5),
        colors="black",
        linewidths=0.1,
        zorder=0,
    )
    ax_dorsal.scatter(
        outside_cells_df["flatmap_x"],
        outside_cells_df["flatmap_y"],
        s=0.5,
        color=color_cells[0],
        alpha=0.1,
        label="Outside cells",
        edgecolors="none",
        rasterized=rasterized,
    )

    ax_dorsal.scatter(
        inside_cells_df["flatmap_x"],
        inside_cells_df["flatmap_y"],
        s=1,
        color=color_cells[1],
        alpha=0.05,
        label="Inside cells",
        edgecolors="none",
        rasterized=rasterized,
    )

    ax_dorsal.set_aspect("equal")
    
    ax_dorsal.axis("off")
    ax_dorsal.invert_xaxis()
    ax_dorsal.invert_yaxis()
    rect = plt.Rectangle((900, 1200), 1000 / atlas_size, height=10, color="k")
    ax_dorsal.add_artist(rect)


if save_fig:
    fig.savefig(save_path / "supplementary_figure_4abc.pdf")

In [ ]:
# Optional 3D plot just for interactive view

try:
    import plotly.graph_objects as go
except ImportError:
    raise ImportError("To run 3D view, plotly must be install")
plot_barcoded = False
# Plot – hull + inside/outside cells
# hull mesh
tri = hull.simplices
mesh = go.Mesh3d(
    x=shifted_pix_coords[:, 0],
    y=shifted_pix_coords[:, 1],
    z=shifted_pix_coords[:, 2],
    i=tri[:, 0],
    j=tri[:, 1],
    k=tri[:, 2],
    opacity=0.45,
    color="lightgrey",
    name="Convex hull",
)

inside_scatter = go.Scatter3d(
    x=inside_cells[:, 0],
    y=inside_cells[:, 1],
    z=inside_cells[:, 2],
    mode="markers",
    marker=dict(size=3, color="red", opacity=0.1),
    name=f"Inside cells ({len(inside_cells)})",
)

outside_scatter = go.Scatter3d(
    x=outside_cells[:, 0],
    y=outside_cells[:, 1],
    z=outside_cells[:, 2],
    mode="markers",
    marker=dict(size=3, color="blue", opacity=0.2),
    name=f"Outside cells ({len(outside_cells)})",
)

if plot_barcoded:
    # --- add a green scatter trace for barcoded rabies cells (not shifted)
    barcoded_scatter = go.Scatter3d(
        x=barcoded_cells[:, 0],
        y=barcoded_cells[:, 1],
        z=barcoded_cells[:, 2],
        mode="markers",
        marker=dict(size=3, color="green", opacity=0.01),
        name=f"Barcoded cells ({len(barcoded_cells)})",
    )
    fig = go.Figure(data=[mesh, inside_scatter, outside_scatter, barcoded_scatter])
else:
    fig = go.Figure(data=[mesh, inside_scatter, outside_scatter])
fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z", aspectmode="data"),
    height=800,
    margin=dict(l=0, r=0, t=10, b=0),
)

fig.show()